In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import sys
import copy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
import pandas as pd
import pickle



# 하이퍼파라미터 설정
batch_size = 100
epochs = 10
lr = 0.01
PCA_dim = 8
CLS_num = 2



with open('./data/data.pkl','rb') as file:
    data = pickle.load(file)
X = data['X']
y = data['Y']



x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



def Fit_to_quantum(X,PCA_dim):
    pca = PCA(n_components=PCA_dim)
    X_pca = pca.fit_transform(X)
    return X_pca
    


# 정규화 (표준 스케일러 사용)
#x_train_pca = Fit_to_quantum(x_train,PCA_dim)
#x_test_pca = Fit_to_quantum(x_test,PCA_dim)

# PyTorch Tensor로 변환
x_train_pca, y_train = torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
x_test_pca, y_test = torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)


class Feature_data_loader(Dataset):
    def __init__(self,x_train,y_train):
        self.feature1 = x_train
        temp = copy.deepcopy(x_train)
        shuffle = torch.randperm(len(temp))
        self.feature2 = temp[shuffle]
        
        self.y1 = y_train
        temp_y = copy.deepcopy(y_train)
        self.y2 = temp_y[shuffle]
        
    def __len__(self):
        return len(self.feature1)
    def __getitem__(self,idx):
        input1 = self.feature1[idx]
        input2 = self.feature2[idx]
        if self.y1[idx] == self.y2[idx]:
            label = torch.tensor(1.).float()
        else:
            label = torch.tensor(0.).float()
        return [input1,input2],label


# DataLoader 생성


train_loader = DataLoader(TensorDataset(x_train_pca, y_train.float()), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_pca, y_test.float()), batch_size=batch_size, shuffle=False)


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import OrderedDict
import math
from kan import KAN
from functions.training import Early_stop_train




class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cls_layer_1 = nn.Linear(PCA_dim,PCA_dim*PCA_dim)
        self.cls_layer_2 = nn.Linear(PCA_dim*PCA_dim,PCA_dim*2-1)
        self.output_layer = nn.Linear(2*PCA_dim-1,1)
    def forward(self, x):
        x = self.cls_layer_1(x)
        x = nn.ReLU()(x)
        x = self.cls_layer_2(x)
        x = nn.ReLU()(x)
        output = self.output_layer(x)
        output = nn.Sigmoid()(output)
        output = torch.squeeze(output)
        return output
# 모델, 손실 함수, 최적화 설정



model = Model(); criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)




train_process = Early_stop_train(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=5000,res=15)

_,acc = train_process.test(test_loader)
print(f"Test Accuracy: {acc:.2f}")




Test set: Average loss: 1.3843, Accuracy: 151.9999988079071/300 (51%)
Epoch 1 Loss 0.692567 acc : 0.526291 stop count : 0
Test set: Average loss: 1.3802, Accuracy: 157.9999988079071/300 (53%)
Epoch 2 Loss 0.689730 acc : 0.549202 stop count : 0
Test set: Average loss: 1.3762, Accuracy: 159.9999988079071/300 (53%)
Epoch 3 Loss 0.688009 acc : 0.554438 stop count : 0
Test set: Average loss: 1.3723, Accuracy: 162.0000011920929/300 (54%)
Epoch 4 Loss 0.685517 acc : 0.577848 stop count : 0
Test set: Average loss: 1.3684, Accuracy: 167.00000023841858/300 (56%)
Epoch 5 Loss 0.683422 acc : 0.601978 stop count : 0
Test set: Average loss: 1.3644, Accuracy: 171.00000023841858/300 (57%)
Epoch 6 Loss 0.681598 acc : 0.617132 stop count : 0
Test set: Average loss: 1.3605, Accuracy: 175.00000023841858/300 (58%)
Epoch 7 Loss 0.678631 acc : 0.631898 stop count : 0
Test set: Average loss: 1.3566, Accuracy: 181.00000023841858/300 (60%)
Epoch 8 Loss 0.677129 acc : 0.628131 stop count : 0
Test set: Average l

In [21]:
from kan import KAN
from functions.training import Early_stop_train_KAN
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.KAN = KAN([PCA_dim,PCA_dim*2-1,1],grid=5)
    def forward(self, x):
        output = self.KAN(x)
        output = nn.Sigmoid()(output)
        output = torch.squeeze(output)
        return output


model = Model(); criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)




train_process = Early_stop_train_KAN(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=50,res=15,lamb=0.005)

_,acc = train_process.test(test_loader)
print(f"Test Accuracy: {acc:.2f}")


Test set: Average loss: 2.5148, Accuracy: 152.99999713897705/300 (51%)
Epoch 1 Loss 0.811853 acc : 0.665714 reg : 0.172128 stop count : 0
Test set: Average loss: 1.5558, Accuracy: 222.00000286102295/300 (74%)
Epoch 2 Loss 0.629140 acc : 0.817143 reg : 0.178508 stop count : 0
Test set: Average loss: 1.1466, Accuracy: 255.00000715255737/300 (85%)
Epoch 3 Loss 0.574807 acc : 0.830000 reg : 0.174328 stop count : 0
Test set: Average loss: 1.0163, Accuracy: 255.00000715255737/300 (85%)
Epoch 4 Loss 0.530724 acc : 0.848571 reg : 0.181639 stop count : 0
Test set: Average loss: 1.0462, Accuracy: 248.99999499320984/300 (83%)
Epoch 5 Loss 0.515780 acc : 0.864286 reg : 0.175536 stop count : 1
Test set: Average loss: 0.9599, Accuracy: 255.00000715255737/300 (85%)
Epoch 6 Loss 0.489444 acc : 0.862857 reg : 0.177121 stop count : 0
Test set: Average loss: 0.9354, Accuracy: 258.0000042915344/300 (86%)
Epoch 7 Loss 0.461819 acc : 0.885714 reg : 0.176823 stop count : 0
Test set: Average loss: 0.9544, Ac

KeyboardInterrupt: 